
## **Livrables Attendus**
- Notebook Python complet avec code commenté
- Visualisations claires pour chaque étape d'analyse
- Rapport d'interprétation des clusters
- Plan d'actions marketing par segment
- Présentation synthétique des résultats

In [116]:
#importation de packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Question 1.1 :** Quelles sont les premières lignes du dataset et sa structure générale ? Combien de transactions et de clients uniques contient-il initialement ?

In [117]:
#chargement et affichage des données
Data=pd.read_excel("Online Retail.xlsx")
Data.head(10) 


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [118]:
#
Data.isna() #affichage des valeurs manquantes
Data=Data.dropna(subset='CustomerID') #suppression des lignes avec des valeurs manquantes dans la colonne CustomerID
Data.isna().sum() #vérification des valeurs manquantes


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [119]:
Data.shape #affichage de la taille du dataset après suppression des valeurs manquantes


(406829, 8)


**Question 1.3 :** Comment calculer le montant total par transaction ? Quel impact cela aura-t-il sur l'analyse RFM ?


In [120]:
total=Data['Quantity']*Data['UnitPrice']
print(total) #affichage du total des achats par ligne

#L’analyse RFM (Récence, Fréquence, Montant) repose sur trois dimensions :
#Récence (R) : date de la dernière transaction
#Fréquence (F) : nombre de transactions sur une période
#Montant (M) : valeur monétaire totale des transactions
#Calculer correctement le montant total par transaction est crucial pour la dimension M:
#Si on ne calcule pas le montant total par transaction, on risque de sous-estimer ou surestimer la valeur du client
#Cela permet de distinguer :Clients à forte valeur,clients réguliers, et les clients faibles 

{}

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Length: 406829, dtype: float64


{}


### **Étape 2 : Calcul des Métriques RFM**

**Question 2.1 :** Quelle date de référence doit-on utiliser pour le calcul de la récence ? Pourquoi est-il important d'ajouter un jour à la date maximale ?

In [121]:
Recence=Data.groupby('CustomerID')['InvoiceDate'].max().reset_index()
Recence['Recence']= (Recence['InvoiceDate'].max()-Recence['InvoiceDate']).dt.days
Recence=Recence[['CustomerID','Recence']]
Recence
#calcul de la frequence
Frequence=Data.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
Frequence.columns=['CustomerID','Frequence']
Frequence
#calcul du montant  total
Montant=Data.groupby('CustomerID').apply(lambda x: np.sum(x['Quantity']*x['UnitPrice'])).reset_index()
Montant.columns=['CustomerID','Montant']
Montant
#fusion des trois dimensions RFM
RFM=Recence.merge(Frequence,on='CustomerID').merge(Montant,on='CustomerID')
RFM.head()


C:\Users\lamin\AppData\Local\Temp\ipykernel_31136\3667452353.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Montant=Data.groupby('CustomerID').apply(lambda x: np.sum(x['Quantity']*x['UnitPrice'])).reset_index()


,CustomerID,Recence,Frequence,Montant
0,12346.0,325,2,0.00
1,12347.0,1,7,4310.00
2,12348.0,74,4,1797.24
3,12349.0,18,1,1757.55
4,12350.0,309,1,334.40


**Question 2.3 :** Combien de clients sont finalement inclus dans l'analyse RFM après nettoyage ?

In [122]:

RFM[['Recence','Frequence','Montant']].describe() #affichage des statistiques descriptives des données RFM

,Recence,Frequence,Montant
count,4372.000000,4372.000000,4372.000000
mean,91.047118,5.075480,1898.459701
std,100.765435,9.338754,8219.345141
min,0.000000,1.000000,-4287.630000
25%,16.000000,1.000000,293.362500
50%,49.000000,3.000000,648.075000
75%,142.000000,5.000000,1611.725000
max,373.000000,248.000000,279489.020000



### **Étape 3 : Analyse des Statistiques Descriptives RFM**

**Question 3.1 :** Quelles sont les statistiques descriptives (moyenne, médiane, écart-type) pour chaque métrique RFM ?

In [123]:
#Calcul de la moyenne 
RFM_mean=RFM[['Recence','Frequence','Montant']].mean()
RFM_mean

Recence        91.047118
Frequence       5.075480
Montant      1898.459701
dtype: float64

In [124]:
#calcule de la mediane
RFM_median=RFM[['Recence','Frequence','Montant']].median()
RFM_median

Recence       49.000
Frequence      3.000
Montant      648.075
dtype: float64

In [125]:
#calcule de l'écart type
RFM_std=RFM[['Recence','Frequence','Montant']].std()
RFM_std

Recence       100.765435
Frequence       9.338754
Montant      8219.345141
dtype: float64

**Question 3.2 :** Que nous apprend l'asymétrie (skewness) et l'aplatissement (kurtosis) des distributions sur le comportement des clients ?

In [126]:
RFM[['Recence','Frequence','Montant']].skew

'mesure la symétrie d’une distribution par rapport à sa moyenne.'
' Une distribution symétrique a une skewness de 0.'
' Une skewness positive indique une queue plus longue à droite, '
'tandis qu’une skewness négative indique une queue plus longue à gauche.'

'tandis qu’une skewness négative indique une queue plus longue à gauche.'

In [127]:
RFM[['Recence','Frequence','Montant']].kurtosis

'mesure la “hauteur” et la concentration de la distribution autour de la moyenne.'


'mesure la “hauteur” et la concentration de la distribution autour de la moyenne.'

### **Étape 4 : Détection et Analyse des Outliers**

**Question 4.1 :** Quelle méthode statistique utiliser pour détecter les outliers ? Pourquoi choisir la méthode IQR plutôt que le Z-score dans ce contexte ?

### **Étape 6 : Matrice de Corrélation**

**Question 6.1 :** Existe-t-il des corrélations entre les métriques RFM ? Lesquelles sont significatives ?


In [128]:
#correlation entre les variables RFM
RFM[['Recence','Frequence','Montant']].corr()

,Recence,Frequence,Montant
Recence,1.000000,-0.259342,-0.132001
Frequence,-0.259342,1.000000,0.566122
Montant,-0.132001,0.566122,1.000000



**Question 6.2 :** Comment interpréter la corrélation entre fréquence et montant ? Est-elle attendue ?


In [129]:
#la correlation est entre la frequence et le montant est égale car on a respectivement 1.000000 et 0.566122
#Oui elle est attendue car plus un achete plus montant augmente


**Question 6.3 :** La récence est-elle corrélée avec les autres métriques ? Qu'indique cette relation sur le comportement client ?

In [130]:
#la recence et la frequence ont une correlation négative comme elle correspont a la date de la dernière transaction
#Oui elle est attendue car plus la recence est faible plus la frequence est élevée

**Question 6.4 :** Comment ces corrélations impactent-elles le clustering ? Faut-il en tenir compte dans le choix des algorithmes ?

In [131]:
#oui la correlation impacte le clustering car elle permet de mieux comprendre les relations entre les variables RFM
#et d'identifier des segments de clients similaires pour un marketing ciblé.

### **Étape 7 : Normalisation et Préparation pour Clustering**

**Question 7.1 :** Pourquoi est-il nécessaire de normaliser les données avant le clustering ? Quels problèmes éviterait-on ?